# Text mining & Search Project

### Università degli Studi di Milano-Bicocca  2020/2021

**Luzzi Federico** 816753 **Peracchi Marco** 800578

# Text Processing & Representation

In [1]:
# Base libraries
import nltk
import pandas as pd
import numpy as np
import re
import os
import string
import matplotlib.pyplot as plt
import sklearn
from wordcloud import WordCloud

In [2]:
# Libraries for text tokenization
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import  WordPunctTokenizer
from nltk.tokenize import  BlanklineTokenizer

In [3]:
# Libraries for stemming and lemmatization
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [4]:
# Libraries for text representation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import save_npz, load_npz # save and load spase matrix
from gensim.models.doc2vec import Doc2Vec, TaggedDocument # doc2vec

In [5]:
# Download of necessary contents
nltk.download("stopwords")
stop_words = nltk.corpus.stopwords.words("english")
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Marco\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Marco\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Marco\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
# Import dataset
df = pd.read_csv("data/labeled_data.csv", sep = ',').drop("Unnamed: 0", axis=1)
df.head()

,count,hate_speech,offensive_language,neither,class,tweet
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [7]:
# Example text
text = df["tweet"][19999]
text

"RT @shakiraevanss: Criticize Amanda for saying the n word, sure, but don't make jokes about her sexual assault, don't be trash."

### Preprocessing

In [8]:
def preprocessing(text):
    text = text.lower() # Lowering case
    remove_url = re.sub(r'(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})', ' ', text) # Removing url
    remove_retweet = re.sub(r"@\w+", " ",remove_url) # Removing retweet
    remove_retweet = re.sub(r"&\w+", " ",remove_retweet) # Remove &amp
    remove_retweet = re.sub(r"\b([!#\$%&\\\(\)\*\+,-\./:;<=>\?@\[\]\^_`\{|\}\"~]+)\b", " ",remove_retweet) # Must check this one
    remove_retweet = re.sub(r"([a-z])\1{3,}", r"\1",remove_retweet)
    remove_punc = remove_retweet.translate(str.maketrans('', '', string.punctuation)) # Remove punctuation
    final_text = re.sub(r'\d+', ' ', remove_punc) # Remove number 
    final_text = re.sub(r'\s+', ' ', final_text) # Removing exceeding spaces
    return final_text

In [9]:
text_prep = preprocessing(text)
text_prep

'rt criticize amanda for saying the n word sure but dont make jokes about her sexual assault dont be trash'

### Tokenization

In [10]:
def tokenization(text_clean, tok = "tweet"):
    if tok == "tweet": # TweetTokenizer
        tt = TweetTokenizer()
        tokenized_text = tt.tokenize(text_clean)
    elif tok == "wordpunct": # WordPunctTokenizer
        wpt = WordPunctTokenizer()
        tokenized_text = wpt.tokenize(text_clean)
    return tokenized_text

In [11]:
text_tok = tokenization(text_prep)
print(text_tok)

['rt', 'criticize', 'amanda', 'for', 'saying', 'the', 'n', 'word', 'sure', 'but', 'dont', 'make', 'jokes', 'about', 'her', 'sexual', 'assault', 'dont', 'be', 'trash']


### Removing stopwords

In [12]:
def remove_stopwords(tokenized_text):
    remove_sw = []
    for token in tokenized_text:
        stop_words.append("rt") # Added a stop words, RT of Retweet
        if token.lower() not in stop_words:
             remove_sw.append(token)
    return remove_sw

In [13]:
text_sw = remove_stopwords(text_tok)
print(text_sw)

['criticize', 'amanda', 'saying', 'n', 'word', 'sure', 'dont', 'make', 'jokes', 'sexual', 'assault', 'dont', 'trash']


### Stemming

In [14]:
def stemmer(tokenized_text):
    porter = PorterStemmer()
    return [porter.stem(word) for word in tokenized_text]

In [15]:
print(stemmer(text_sw))

['critic', 'amanda', 'say', 'n', 'word', 'sure', 'dont', 'make', 'joke', 'sexual', 'assault', 'dont', 'trash']


### Lemmatization

In [16]:
# pos-tagging (1 document)
def pos_tagging(doc_token):
    return nltk.pos_tag(doc_token)

# convertion of pos tagging
def get_wordnet_pos(word_tag):
    if word_tag.startswith('J'):
        return "a"
    elif word_tag.startswith('V'):
        return "v"
    elif word_tag.startswith('R'):
        return "r"
    else:
        return "n"
    
# lemmatizer one word 
def lemmatizer(word):
    pos = get_wordnet_pos(word[1])
    wnl = WordNetLemmatizer()
    return wnl.lemmatize(word[0], pos = pos)

# lemmatizer one document
def lemmatizer_doc(doc_token):
    lemmas = [] 
    
    pos_document = pos_tagging(doc_token) # pos tagging
    for token in pos_document:
        lemmas.append( lemmatizer(token) ) # lemmatization x word
    
    return lemmas

In [17]:
print(lemmatizer_doc(text_sw))

['criticize', 'amanda', 'say', 'n', 'word', 'sure', 'dont', 'make', 'joke', 'sexual', 'assault', 'dont', 'trash']


### Processing everything on dataset
The following cell applies:

1. Preprocessing
2. Tokenization
3. Stopwords removal
4. Lemmatization
5. Save the result

In [18]:
# Sum up function
def processing(text):
    text_prep = preprocessing(text)
    text_prep = tokenization(text_prep)
    text_prep = remove_stopwords(text_prep)
    #text_prep = lemmatizer_doc(text_prep)
    text_prep = stemmer(text_prep)
    text_prep = " ".join(text_prep)
    return text_prep

In [19]:
# Apply on all text
df["tweet_clean"] = df["tweet"].apply(lambda x : processing(x))

In [20]:
df.head()

,count,hate_speech,offensive_language,neither,class,tweet,tweet_clean
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,woman shouldnt complain clean hous man alway t...
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,boy dat cold tyga dwn bad cuffin dat hoe st place
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,dawg ever fuck bitch start cri confus shit
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,look like tranni
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,shit hear might true might faker bitch told ya


In [21]:
# Save dataset
df.to_csv("data/processed_data.csv", index = False)

# Text Representation

In [6]:
# Import dataset
if 'processed_data.csv' not in os.listdir('data'):
    print("!!! ERROR !!!\n --- Before this, previous cells ---")
else:
    df = pd.read_csv("data/processed_data.csv", sep = ",")
    print("Load & preprocessing --- DONE")

Load & preprocessing --- DONE


In [7]:
# drop tweet list NA
df["tweet_list"] = df["tweet_clean"].str.split(" ").tolist()
df.dropna(inplace = True)
df = df.reset_index(drop = True)# drop 2 NA because tweet is empty

In [8]:
corpus = df["tweet_clean"]

### Bag of words

In [25]:
# Using CountVectorizer
vectorizer = CountVectorizer(binary = True)
X = vectorizer.fit_transform(corpus)

# Shape
X.toarray().shape

(24781, 15163)

In [26]:
# Example of presence of a word
X.toarray()[1][X.toarray()[1] == 1]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [27]:
# Save bag of words
save_npz('data/representations/bag_of_words.npz', X)

In [28]:
# Load bag of words
bag_of_words = load_npz('data/representations/bag_of_words.npz')

###  Count Vector

In [29]:
# Using CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

# Example of presence of a word
X.toarray()[1][X.toarray()[1] == 2]

array([2], dtype=int64)

In [31]:
# Save count Vector
save_npz('data/representations/count_vector.npz', X)

### Tf-idf

In [9]:
vectorizer = TfidfVectorizer(
    use_idf=True,
    max_features=10000,
    min_df=5,
    max_df=0.501
    )
tfidf = vectorizer.fit_transform(corpus).toarray()

In [10]:
tfidf.shape

(24781, 3614)

In [11]:
np.save('data/representations/tf-idf.npy', tfidf)

### Doc2Vec

In [35]:
# Generate model of vectorization
if 'd2v.model' not in os.listdir('models/doc2vec'):
    documents = [TaggedDocument (doc, [i]) for i, doc in enumerate(df["tweet_list"])]
    model = Doc2Vec (documents, vector_size=100, window=10, min_count=1)
    model.save("models/doc2vec/d2v.model")
else:
    model = Doc2Vec.load("models/doc2vec/d2v.model")
print("Doc2Vec model --- DONE")

Doc2Vec model --- DONE


In [36]:
# Vectorization
doc2vec = df["tweet_list"].apply(lambda x: model.infer_vector(x))
print("Vectorization --- DONE")

Vectorization --- DONE


In [37]:
doc2vec[0]

array([-0.00231703,  0.00256387, -0.00139071, ...,  0.00306892,
       -0.00538458,  0.00151551], dtype=float32)

In [38]:
np.save('data/representations/doc2vec.npy', doc2vec)